# GERAÇÃO DE MAPAS BDGD

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
from os import listdir
import zipfile
import time
import os 
from datetime import datetime
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl, Popup
from shapely.ops import unary_union
import openpyxl
import glob
import pickle

# Funções

In [2]:
def read_mult_zip(tbl, dir, lstorg):
    DS = pd.DataFrame()
    lst = [i for i in lstorg if tbl in i]
    for file in lst:
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        print('####### ' + file  + ' ####### ' + now +' ####### ')
        filezip = 'zip://'+ dir +  arq + '!'+ file
        df = gpd.read_file(filezip)
        df['file'] = file
        DS = pd.concat([DS, df], ignore_index = True)
    
    return DS

def agg_ano(df, IniNomeVar, funtion = 'sum'):
    lst_var = [i for i in df.columns if IniNomeVar in i]
    fun_switcher = {
        'sum': df[lst_var].sum(axis =1),
        'count': df[lst_var].count(axis =1),
        'mean': df[lst_var].mean(axis =1) 
    }
    
    return fun_switcher.get(funtion, np.nan)

### extract 7zip files
def extract_7zip(dir, file):
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print('####### Extracting: ' + file  + ' ####### ' + now +' ####### ')
    target_archive =  multivolumefile.open(dir +file, mode='rb')
    file7Zip =py7zr.SevenZipFile(target_archive, 'r')
    file7Zip.extractall(path=dir)
    file7Zip.close
    return True
 

In [3]:
    # Mapas
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Maps'
    ),
    'Google Satellite': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite'
    ),
    'Google Terrain': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Terrain'
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite'
    ),
    'Esri Satellite': folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ),
    'CartoDB positron': folium.TileLayer(
        tiles='CartoDB positron',
        attr='CartoDB',
        name='Light Map'
    )
}

# Dicionários

In [4]:
dic_TCABOGTCEOM = {'0':'Não informado',
                     '1': 'Aérea Compacta',
                     '2': 'Aérea Concêntrica',
                     '3': 'Aérea Horizontal',
                     '4': 'Aérea Multiplex',
                     '5': 'Aérea Triangular',
                     '6': 'Aérea Vertical',
                     '7': 'Subterrânea',
                     '8': 'Submersa',
                     '9': 'Interna',
                     '10': 'Em Subestação'}

dic_FAS_CON = {'0':'Não informado',
                 'ABCN': 'trifasico',
                 'ABC': 'trifasico',
                 'ABN': 'bifasico',
                 'BCN': 'bifasico',
                 'CAN': 'bifasico',
                 'AX': 'monofasico',
                 'BX': 'monofasico',
                 'CX': 'monofasico',
                 'AB': 'bifasico',
                 'BC': 'bifasico',
                 'CA': 'bifasico',
                 'AN': 'monofasico',
                 'BN': 'monofasico',
                 'CN': 'monofasico',
                 'A': 'monofasico',
                 'B': 'monofasico',
                 'C': 'monofasico',
                 'N': 'apenas_neutro'}

dic_FORM_CAB = {'0':'Não informado',
                 '1': 'Singelo',
                 '2': 'Multiplexado Duplex',
                 '3': 'Multiplexado Triplex',
                 '4': 'Multiplexado Quadruplex',
                 '5': 'Interno de subestação',
                 '6': 'Concêntrico Monofásico',
                 '7': 'Concêntrico Bifásico',
                 '8': 'Concêntrico Trifásico'}

dic_A3_FAS = {'01':'Nu',
                 '02': 'Isolado',
                 '03': 'Isolado',
                 '04': 'Isolado',
                 '05': 'Isolado',
                 '06': 'Protegido',
                 '07': 'Protegido',
                 '08': 'Protegido',
                 '09': 'Isolado'}

dic_ISO_FAS = {'0':'Não informado',
                 '2': 'Coberto',
                 '4': 'Encapado',
                 '5': 'EPR/PVC',
                 '6': 'EPR/XLPE',
                 '7': 'Isolado - EPR',
                 '8': 'Isolado - Papel impregnado',
                 '13': 'Isolado - PE',
                 '9': 'Isolado - PVC',
                 '10': 'Isolado - XLPE',
                 '11': 'Misto OPGW',
                 '12': 'Nu',
                 '15': 'Protegido – EPR',
                 '14': 'Protegido – PVC',
                 '17': 'Protegido – XLPE'}

dic_TPOTAPRT = {
            '0' : 0,
            '1' : 3,
            '2' : 5,
            '3' : 10,
            '4' : 15,
            '5' : 20,
            '6' : 22.5,
            '7' : 25,
            '8' : 30,
            '9' : 35,
            '10' : 37.5,
            '11' : 38.1,
            '12' : 40,
            '13' : 45,
            '14' : 50,
            '15' : 60,
            '16' : 75,
            '17' : 76.2,
            '18' : 88,
            '19' : 100,
            '20' : 112.5,
            '21' : 114.3,
            '22' : 120,
            '23' : 138,
            '24' : 150,
            '25' : 167,
            '26' : 175,
            '27' : 180,
            '28' : 200,
            '29' : 207,
            '30' : 225,
            '31' : 250,
            '32' : 276,
            '33' : 288,
            '34' : 300,
            '35' : 332,
            '36' : 333,
            '37' : 400,
            '38' : 414,
            '39' : 432,
            '40' : 500,
            '41' : 509,
            '42' : 667,
            '43' : 750,
            '44' : 833,
            '45' : 1000,
            '46' : 1250,
            '47' : 1300,
            '48' : 1500,
            '49' : 1750,
            '50' : 2000,
            '51' : 2250,
            '52' : 2300,
            '53' : 2400,
            '54' : 2500,
            '55' : 2750,
            '56' : 2900,
            '57' : 3000,
            '58' : 3125,
            '59' : 3300,
            '60' : 3750,
            '61' : 4000,
            '62' : 4200,
            '63' : 4500,
            '64' : 5000,
            '65' : 6250,
            '66' : 6500,
            '67' : 7000,
            '68' : 7500,
            '69' : 7800,
            '70' : 8000,
            '71' : 9000,
            '72' : 9375,
            '73' : 9600,
            '74' : 10000,
            '75' : 12000,
            '76' : 12500,
            '77' : 13300,
            '78' : 15000,
            '79' : 16000,
            '80' : 18000,
            '81' : 18750,
            '82' : 20000,
            '83' : 25000,
            '84' : 26000,
            '85' : 26600,
            '86' : 28000,
            '87' : 30000,
            '88' : 32000,
            '89' : 33000,
            '90' : 33300,
            '91' : 40000,
            '92' : 45000,
            '93' : 50000,
            '94' : 60000,
            '95' : 67000,
            '96' : 75000,
            '97' : 80000,
            '98' : 83000,
            '99' : 85000,
            '100' : 90000,
            '101' : 100000,
            '102' : 200000,
            '107' : 300000,
            '108' : 400000,
            '109' : 500000,
            '103' : 14550000,
            '104' : 17320000,
            '105' : 19100000,
            '106' : 41550000}



dic_TCLASUBCLA =  { 'Não informado':'0',
                    'Residencial':'RE1',
                    'Residencial baixa renda':'RE2',
                    'Residencial baixa renda indígena':'RE3',
                    'Residencial baixa renda quilombola':'RE4',
                    'Residencial baixa renda benefício de prestação continuada da assistência social – BPC':'RE5',
                    'Residencial baixa renda multifamiliar':'RE6',
                    'Industrial':'IN',
                    'Comercial':'CO1',
                    'Serviços de transporte, exceto tração elétrica':'CO2',
                    'Serviços de comunicações e telecomunicações':'CO3',
                    'Associação e entidades filantrópicas':'CO4',
                    'Templos religiosos':'CO5',
                    'Administração condominial :  iluminação e instalações de uso comum de prédio ou conjunto de edificações':'CO6',
                    'Iluminação em rodovias :  solicitada por quem detenha concessão ou autorização para administração em rodovias':'CO7',
                    'Semáforos, radares e câmeras de monitoramento de trânsito, solicitados por quem detenha concessão ou autorização para controle de trânsito':'CO8',
                    'Outros serviços e outras atividades':'CO9',
                    'Agropecuária rural':'RU1',
                    'Agropecuária rural (poços de captação de água,para atender finalidades de que trata este inciso, desde que não haja comercialização da água)':'RU1A',
                    'Agropecuária rural (serviço de bombeamento de água destinada à atividade de irrigação)':'RU1B',
                    'Agropecuária urbana':'RU2',
                    'Residencial rural':'RU3',
                    'Cooperativa de eletrificação rural':'RU4',
                    'Agroindustrial':'RU5',
                    'Serviço público de irrigação rural':'RU6',
                    'Escola agrotécnica':'RU7',
                    'Aquicultura':'RU8',
                    'Poder público federal':'PP1',
                    'Poder público estadual ou distrital':'PP2',
                    'Poder público municipal':'PP3',
                    'Iluminação pública':'IP',
                    'Tração elétrica':'SP1',
                    'Água, esgoto e saneamento':'SP2',
                    'Consumo próprio pela distribuidora':'CPR',
                    'Consumo próprio pela distribuidora para estação de recarga de veículos elétricos':'CPRVE',
                    'Concessionária ou Permissionária':'CSPS'
}


# Define pastas e arquivos

In [5]:
pasta_bdgd = 'E:/baseedpsp/Arquivos BDGD/'
arq ='391_2023-12-31_V11_20240226-0037.zip'
ano = 2023

In [6]:
pasta_apoio = "C:/GIT/ted_aneel/Mapas BDGD/resultados/"

In [7]:
# Procura arquivos 

lst_zip = [f for f in listdir(pasta_bdgd) if 'zip' in f ]


In [8]:
arqzip= zipfile.ZipFile(pasta_bdgd +arq)
lst_shp =[i for i in arqzip.namelist() if (('shp' in i) and not ('.shp.xml' in i))]
lst_dbf =[i for i in arqzip.namelist() if ('dbf' in i)]

In [9]:
lst_dbf

['391_2023-12-31_V11_20240226-0037/ARAT.dbf',
 '391_2023-12-31_V11_20240226-0037/BAR.dbf',
 '391_2023-12-31_V11_20240226-0037/BAR.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/BASE.dbf',
 '391_2023-12-31_V11_20240226-0037/BASE.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/BAY.dbf',
 '391_2023-12-31_V11_20240226-0037/BAY.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/CONJ.dbf',
 '391_2023-12-31_V11_20240226-0037/CTAT.dbf',
 '391_2023-12-31_V11_20240226-0037/CTAT.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/CTMT.dbf',
 '391_2023-12-31_V11_20240226-0037/CTMT.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/EQCR.dbf',
 '391_2023-12-31_V11_20240226-0037/EQCR.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/EQME.dbf',
 '391_2023-12-31_V11_20240226-0037/EQME.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/EQRE.dbf',
 '391_2023-12-31_V11_20240226-0037/EQRE.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/EQSE.dbf',
 '391_2023-12-31_V11_20240226-0037/EQSE.dbf.xml',
 '391_2023-12-31_V11_20240226-0037/EQTRAT.dbf',


# UC

In [10]:
### UCBT 

tbl = 'UCBT'
DS = read_mult_zip(tbl,pasta_bdgd, lst_shp)
DS['ENE']=agg_ano(DS, 'ENE', 'sum')
DS['DIC']=agg_ano(DS, 'DIC', 'sum')
DS['FIC']=agg_ano(DS, 'FIC', 'sum')

UCBT = DS
UCBT['Tensao']= 'BT'
UCBT=UCBT.to_crs('EPSG:4674')

####### 391_2023-12-31_V11_20240226-0037/UCBT_01.shp ####### 29/03/2024 10:32:27 ####### 
####### 391_2023-12-31_V11_20240226-0037/UCBT_02.shp ####### 29/03/2024 11:08:57 ####### 
####### 391_2023-12-31_V11_20240226-0037/UCBT_03.shp ####### 29/03/2024 11:54:43 ####### 
####### 391_2023-12-31_V11_20240226-0037/UCBT_04.shp ####### 29/03/2024 12:43:12 ####### 


In [14]:
UCBT.head()

,COD_ID,DIST,PAC,RAMAL,PN_CON,UNI_TR_MT,CTMT,UNI_TR_AT,SUB,CONJ,...,TIP_SIST,DATA_BASE,POINT_X,POINT_Y,geometry,file,ENE,DIC,FIC,Tensao
0,36077101,391,BT1000106603,1000106603,9065270,045ET000868869,RVHE0104,045TRVHE00019,VHE,16656,...,RD_INTERLIG,2023-12-31,-46.557472,-23.502350,POINT (-46.55747 -23.50235),391_2023-12-31_V11_20240226-0037/UCBT_01.shp,2299.277857,0.80,2.0,BT
1,151277142,391,BT1000282454,1000282454,8799008,045ET000243399,RVHE0105,045TRVHE00039,VHE,16656,...,RD_INTERLIG,2023-12-31,-46.557765,-23.498990,POINT (-46.55777 -23.49899),391_2023-12-31_V11_20240226-0037/UCBT_01.shp,528.747893,8.35,6.0,BT
2,36362743,391,BT1000100093,1000100093,8745424,045ET000542699,RVHE0105,045TRVHE00039,VHE,16656,...,RD_INTERLIG,2023-12-31,-46.557809,-23.495427,POINT (-46.55781 -23.49543),391_2023-12-31_V11_20240226-0037/UCBT_01.shp,1583.486697,7.98,3.0,BT
3,150876927,391,BT1001486970,1001486970,9333268,045ET000685589,RVHE0105,045TRVHE00039,VHE,16656,...,RD_INTERLIG,2023-12-31,-46.558207,-23.495197,POINT (-46.55821 -23.49520),391_2023-12-31_V11_20240226-0037/UCBT_01.shp,920.389226,7.98,3.0,BT
4,150192823,391,BT1000115982,1000115982,9065282,045ET000864269,RVHE0104,045TRVHE00019,VHE,16656,...,RD_INTERLIG,2023-12-31,-46.557422,-23.505399,POINT (-46.55742 -23.50540),391_2023-12-31_V11_20240226-0037/UCBT_01.shp,541.000000,2.15,1.0,BT


In [15]:
# A linha abaixo visa gerar um arquivo .pkl para posterior manipulação sem a necessidade de carregar novamente da BDGD

with open(pasta_bdgd + 'ucbt.pkl', 'wb') as arquivo:
    pickle.dump(UCBT, arquivo)


In [35]:
UCBT.head()

,COD_ID,DIST,PAC,RAMAL,PN_CON,UNI_TR_MT,CTMT,UNI_TR_AT,SUB,CONJ,...,FIC_12,SEMRED,DESCR,TIP_SIST,geometry,file,ENE,DIC,FIC,Tensao
0,150534139,391,BT1001820849,1001820849,None,045ET005247759,RGUL0105,045TRGUL00029,GUL,16639,...,0.0,1,B003461982,RD_INTERLIG,POINT (-46.51537 -23.46330),391_2022-12-31_V11_20230428-2024/UCBT_01.shp,1852.0,0.00,0.0,BT
1,150066710,391,BT1001296792,1001296792,None,045ET005047719,RVGA0113,045TRVGA00029,VGA,16655,...,1.0,1,B002023506,RD_INTERLIG,POINT (-46.53373 -23.47245),391_2022-12-31_V11_20230428-2024/UCBT_01.shp,1843.0,1.05,2.0,BT
2,38342171,391,BT1000282800,1000282800,None,045ET000589549,RVGA0114,045TRVGA00019,VGA,16655,...,0.0,1,B000003498,RD_INTERLIG,POINT (-46.54736 -23.47983),391_2022-12-31_V11_20230428-2024/UCBT_01.shp,1296.0,0.00,0.0,BT
3,150914254,391,BT1002004010,1002004010,None,045ET005533509,RPME0112,045TRPME00029,PME,16646,...,1.0,1,B003759813,RD_INTERLIG,POINT (-46.42151 -23.44560),391_2022-12-31_V11_20230428-2024/UCBT_01.shp,2515.0,1.90,2.0,BT
4,92762239,391,BT1000446779,1000446779,None,165ET000119209,RPED1301,165TRPED00019,PED,16645,...,0.0,1,B000541024,RD_INTERLIG,POINT (-46.31642 -23.42871),391_2022-12-31_V11_20230428-2024/UCBT_01.shp,1909.0,1.66,2.0,BT


In [16]:
### UCMT 
tbl = 'UCMT'
DS = read_mult_zip(tbl, pasta_bdgd, lst_shp)
DS['ENE']=agg_ano(DS, 'ENE', 'sum')
DS['DIC']=agg_ano(DS, 'DIC', 'sum')
DS['FIC']=agg_ano(DS, 'FIC', 'sum')

UCMT = DS
UCMT['Tensao']= 'MT'
UCMT=UCMT.to_crs('EPSG:4674')

####### 391_2023-12-31_V11_20240226-0037/UCMT.shp ####### 29/03/2024 19:51:57 ####### 


In [17]:
# A linha abaixo visa gerar um arquivo .pkl para posterior manipulação sem a necessidade de carregar novamente da BDGD

with open(pasta_bdgd + 'ucMt.pkl', 'wb') as arquivo:
    pickle.dump(UCMT, arquivo)


In [18]:
### UGMT 
tbl = 'UGMT'
DS = read_mult_zip(tbl, pasta_bdgd, lst_shp)
DS['ENE']=agg_ano(DS, 'ENE', 'sum')
DS['DIC']=agg_ano(DS, 'DIC', 'sum')
DS['FIC']=agg_ano(DS, 'FIC', 'sum')

UGMT = DS
UGMT['Tensao']= 'MT'
UGMT=UGMT.to_crs('EPSG:4674')


####### 391_2023-12-31_V11_20240226-0037/UGMT.shp ####### 29/03/2024 19:55:42 ####### 


In [19]:
# A linha abaixo visa gerar um arquivo .pkl para posterior manipulação sem a necessidade de carregar novamente da BDGD

with open(pasta_bdgd + 'ugmt.pkl', 'wb') as arquivo:
    pickle.dump(UGMT, arquivo)


In [20]:
### UGBT 
tbl = 'UGBT'
DS = read_mult_zip(tbl, pasta_bdgd, lst_shp)
DS['ENE']=agg_ano(DS, 'ENE', 'sum')
DS['DIC']=agg_ano(DS, 'DIC', 'sum')
DS['FIC']=agg_ano(DS, 'FIC', 'sum')

UGBT = DS
UGBT['Tensao']= 'MT'
UGBT=UGBT.to_crs('EPSG:4674')

####### 391_2023-12-31_V11_20240226-0037/UGBT.shp ####### 29/03/2024 19:55:57 ####### 


In [21]:
# A linha abaixo visa gerar um arquivo .pkl para posterior manipulação sem a necessidade de carregar novamente da BDGD

with open(pasta_bdgd + 'ugbt.pkl', 'wb') as arquivo:
    pickle.dump(UGBT, arquivo)

# CTMT

In [25]:
tbl = 'CTMT'
DS = read_mult_zip(tbl, pasta_bdgd, lst_dbf)
ctmt = DS

####### 391_2023-12-31_V11_20240226-0037/CTMT.dbf ####### 29/03/2024 20:00:01 ####### 
####### 391_2023-12-31_V11_20240226-0037/CTMT.dbf.xml ####### 29/03/2024 20:00:03 ####### 


DriverError: '/vsizip/E:/baseedpsp/Arquivos BDGD/391_2023-12-31_V11_20240226-0037.zip/391_2023-12-31_V11_20240226-0037/CTMT.dbf.xml' not recognized as a supported file format.

In [23]:
### UGBT 
tbl = 'CTMT'
DS = read_mult_zip(tbl, pasta_bdgd, lst_shp)
CTMT = DS

DriverError: E:/baseedpsp/Arquivos BDGD/CTMT.dbf: No such file or directory

In [24]:
CTMT

""


# Ler Pkl

In [8]:
resultados_ucbt = pd.read_pickle(pasta_apoio + 'ucbt.pkl')
resultados_ugbt = pd.read_pickle(pasta_apoio + 'ugbt.pkl')
resultados_ucmt = pd.read_pickle(pasta_apoio + 'ucMt.pkl')
resultados_ugmt = pd.read_pickle(pasta_apoio + 'ugmt.pkl')